# Importing Librarires and Data

In [318]:
import pandas as pd

In [319]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("adilshamim8/salaries-for-data-science-jobs")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\felip\.cache\kagglehub\datasets\adilshamim8\salaries-for-data-science-jobs\versions\15


In [320]:
df = pd.read_csv(r"C:\Users\felip\.cache\kagglehub\datasets\adilshamim8\salaries-for-data-science-jobs\versions\15\salaries.csv")

In [321]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2025,EX,FT,Head of Data,348516,USD,348516,US,0,US,M
1,2025,EX,FT,Head of Data,232344,USD,232344,US,0,US,M
2,2025,SE,FT,Data Scientist,145400,USD,145400,US,0,US,M
3,2025,SE,FT,Data Scientist,81600,USD,81600,US,0,US,M
4,2025,MI,FT,Engineer,160000,USD,160000,US,100,US,M


# Exploratory Data Analysis

In [322]:
# The table has 151.445  rows and 11 columns, 
df.shape

(151445, 11)

In [323]:
# There isn't any missing values in the dataset, also the data types seems to be right
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151445 entries, 0 to 151444
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   work_year           151445 non-null  int64 
 1   experience_level    151445 non-null  object
 2   employment_type     151445 non-null  object
 3   job_title           151445 non-null  object
 4   salary              151445 non-null  int64 
 5   salary_currency     151445 non-null  object
 6   salary_in_usd       151445 non-null  int64 
 7   employee_residence  151445 non-null  object
 8   remote_ratio        151445 non-null  int64 
 9   company_location    151445 non-null  object
 10  company_size        151445 non-null  object
dtypes: int64(4), object(7)
memory usage: 12.7+ MB


In [324]:
# As usual, there is a perceptible  standart deviation in the salary column
df.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
work_year,151445.0,2024.44,0.67,2020.0,2024.0,2025.0,2025.0,2025.0
salary,151445.0,162837.96,208012.40,14000.0,106000.0,147000.0,199000.0,30400000.0
salary_in_usd,151445.0,157527.46,74150.77,15000.0,105800.0,146100.0,198000.0,800000.0
remote_ratio,151445.0,20.94,40.62,0.0,0.0,0.0,0.0,100.0


In [325]:
''' There is a lot of unique values at the 'job_title' column.
although this is not valid to a regression model, this column is for sure a good feature to predict the 'salary' label.
'''
df['job_title'].unique()

array(['Head of Data', 'Data Scientist', 'Engineer', 'AI Product Lead',
       'AI Engineer', 'Business Intelligence Engineer', 'Manager',
       'Computer Vision Engineer', 'Data Management', 'Product Manager',
       'Data Engineer', 'Machine Learning Engineer', 'Software Engineer',
       'Data Analyst', 'Research Scientist', 'Developer', 'Associate',
       'Computational Scientist', 'Analyst', 'AI Scientist',
       'DataOps Engineer', 'Prompt Engineer', 'Data Governance Engineer',
       'Analytics Engineer', 'AI Governance Specialist',
       'Backend Engineer', 'DevOps Engineer', 'Systems Engineer',
       'Tech Lead', 'Solutions Engineer', 'Product Analyst', 'Researcher',
       'Executive', 'Director', 'Consultant', 'Solutions Architect',
       'Data Reporting Specialist', 'BI Developer', 'Data Manager',
       'Machine Learning Scientist', 'Data Governance',
       'Applied Scientist', 'Data Architect', 'Statistical Programmer',
       'Data Specialist', 'Technical Architec

In [326]:
# Let's start seeking for the most frequent values
df['job_title'].value_counts().head(50)

job_title
Data Scientist                     18751
Software Engineer                  16948
Data Engineer                      16352
Data Analyst                       13779
Engineer                           11004
Machine Learning Engineer           8887
Manager                             7811
Analyst                             5396
Research Scientist                  3460
Product Manager                     2576
Applied Scientist                   2381
Associate                           2379
Data Architect                      2216
Analytics Engineer                  2139
AI Engineer                         2013
Research Engineer                   1737
Consultant                          1363
Data Manager                        1073
Developer                           1040
Data Specialist                      984
Solutions Architect                  976
Business Intelligence Engineer       956
Engineering Manager                  902
Systems Engineer                     854
Archit

'Software Engineer' and 'Manager' really isn't the type of job title that we are looking for, because this isn't data science jobs. so let's filter this

In [327]:
# looking to the most frequent job titles that we are aiming, we will create a list of job titles that we are aiming
df[df['job_title'].str.contains('data scientist|data analyst|data engineer|Machine Learning Engineer|bi |ai ', case=False, na=False)]['job_title'].value_counts().head(30)

job_title
Data Scientist               18751
Data Engineer                16352
Data Analyst                 13779
Machine Learning Engineer     8887
AI Engineer                   2013
BI Analyst                     447
BI Developer                   306
AI Architect                   289
AI Developer                   222
Power BI Developer             151
AI Scientist                   138
BI Engineer                    120
AI Specialist                  108
AI Researcher                  106
AI Lead                         35
AI Product Manager              27
AI Product Owner                26
Business Data Analyst           21
BI Data Analyst                 19
Power BI Administrator          18
AI Solution Director            16
AI Product Lead                 14
Big Data Engineer               14
Applied Data Scientist          13
AI Data Scientist               12
GenAI Architect                 12
AI Software Engineer            11
Lead Data Scientist             11
Power BI S

as we can see, 'ai' job titles really isn't a good reference to predict the salary, due it's low frequency on the dataset, and the amount of different titles with 'ai' in it.

# Manipulating data

In [328]:
# Create the list
relevant_job_titles = ['Data Scientist', 'Data Engineer', 'Data Analyst', 'Machine Learning Engineer', 'Data Architect', 'Business Intelligence Engineer', 'BI Engineer', 'Business Intelligence Analyst', 'BI Analyst']

In [329]:
# lenght of the new df with our desired job titles
lenght_filtered_df = df[df['job_title'].isin(relevant_job_titles)].shape[0]

In [330]:
# absolute difference between the original and the new datasets
diff = df.shape[0] - lenght_filtered_df

In [331]:
# difference in percentage
percentage_diff = (lenght_filtered_df / df.shape[0]) * 100

In [346]:
print(f'The new filtered dataframe has {lenght_filtered_df} rows ({percentage_diff:.2f}% of the original dataframe), loosing {diff} of {df.shape[0]} rows')

The new filtered dataframe has 62174 rows (41.05% of the original dataframe), loosing 89271 of 62174 rows


In [333]:
# Filters the dataframe
df = df[df['job_title'].isin(relevant_job_titles)]

In [334]:
# Grouping 'Bussines Intelligence x' and 'BI x' job titles
def group_bi_professionals(job_title):
    if job_title == 'Business Intelligence Analyst':
        return 'BI Analyst'
    elif job_title == 'Business Intelligence Engineer':
        return 'BI Engineer'
    else:
        return job_title

In [335]:
# Apply the funcion
df['job_title'] = df['job_title'].apply(group_bi_professionals)

In [336]:
# Display the new unique job title values
df['job_title'].unique()

array(['Data Scientist', 'BI Engineer', 'Data Engineer',
       'Machine Learning Engineer', 'Data Analyst', 'Data Architect',
       'BI Analyst'], dtype=object)

In [337]:
# Apply Dummy variables (one hot encoding on 'job_title' column)
job_dummies = pd.get_dummies(df['job_title'], prefix='job', dtype=int)
job_dummies.columns

Index(['job_BI Analyst', 'job_BI Engineer', 'job_Data Analyst',
       'job_Data Architect', 'job_Data Engineer', 'job_Data Scientist',
       'job_Machine Learning Engineer'],
      dtype='object')

In [338]:
# Concatinating the datasets
df = pd.concat([df, job_dummies], axis=1)
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,job_BI Analyst,job_BI Engineer,job_Data Analyst,job_Data Architect,job_Data Engineer,job_Data Scientist,job_Machine Learning Engineer
2,2025,SE,FT,Data Scientist,145400,USD,145400,US,0,US,M,0,0,0,0,0,1,0
3,2025,SE,FT,Data Scientist,81600,USD,81600,US,0,US,M,0,0,0,0,0,1,0
10,2025,SE,FT,BI Engineer,180500,USD,180500,US,0,US,M,0,1,0,0,0,0,0
11,2025,SE,FT,BI Engineer,113000,USD,113000,US,0,US,M,0,1,0,0,0,0,0
30,2025,SE,FT,Data Engineer,200000,USD,200000,US,100,US,M,0,0,0,0,1,0,0


In [339]:
# Droppping the old 'job_title' column
df = df.drop(['job_title'], axis=1)
df.head()

,work_year,experience_level,employment_type,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,job_BI Analyst,job_BI Engineer,job_Data Analyst,job_Data Architect,job_Data Engineer,job_Data Scientist,job_Machine Learning Engineer
2,2025,SE,FT,145400,USD,145400,US,0,US,M,0,0,0,0,0,1,0
3,2025,SE,FT,81600,USD,81600,US,0,US,M,0,0,0,0,0,1,0
10,2025,SE,FT,180500,USD,180500,US,0,US,M,0,1,0,0,0,0,0
11,2025,SE,FT,113000,USD,113000,US,0,US,M,0,1,0,0,0,0,0
30,2025,SE,FT,200000,USD,200000,US,100,US,M,0,0,0,0,1,0,0


## Removing unwanted columns

In [350]:
df = df.drop(['salary', 'salary_currency'], axis=1)